### Welcome! This notebook describes a method of generating inferential decompositions from text as introduced in our EMNLP 2023 paper: [Natural Language Decompositions of Implicit Content Enable Better Text Representations](https://arxiv.org/pdf/2305.14583.pdf)!

We will guide you through the process step-by-step, and provide explanations and code snippets along the way. The method can be broken down into the following steps:

1. **Sample a small number of items from your dataset**: Here, we use a dataset of tweets posted by legislators during the 115, 116 and 117th US Congresses. 
2. **Craft Implicit and Explicit Propositions**: Refer to Appendix 2. of our paper for a description of the instructions used in the paper to craft exemplar poropotitions. We will use the same instructions to craft implicit and explicit propositions for our dataset.
3. **Prompt an LLM with the crafted exemplars**: Here, we will use GPT3.5 Turbo for our experiments. 
4. **Validate**: Confirm that a random sample of the generated decompositions are _plausible_.
5. **Downstream Usage**: Use the decompositions in the target task. 

#### Getting Started

To begin, run the first cell below to import the necessary packages and set up the environment. The helper functions and accompanying code are in `eval_mteb.py` and `generation_utils.py`. 

##### Note: 
We assume that your OPENAI_API_KEY is an environment variable. One way to set it is by running - `conda env config vars set OPENAI_API_KEY=<your_key_here>` inside your conda environment. It can also be set manually in the config by setting  `config["llm"]["openai_api_key"]`.

The code below was tested on a linux server, but should work on other hardware with alterations to pytorch versioning. \[More additions coming soon!]

In [1]:
import os 
import json 
from tqdm import tqdm
import random
from simple_colors import * 

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
from pathlib import Path

from transformers import GenerationConfig 


#### Choosing the data 

For the purpose of this tutorial, we choose a dataset of congressional tweets sampled from the 115th, 116th and 117th Congress. The data can be found in `data/sampled_tweets_senate_115-117.jsonl`.

### Step 1: Sample a small number of items from your dataset

In our case, we sample a small number of tweets from the dataset. 

In [2]:
from misc_utils import read_jsonl, write_jsonl, create_textboxes, show_document

TWEETS_FILEPATH = Path('data/sampled_tweets_senate_115-117.jsonl')
tweets = read_jsonl(TWEETS_FILEPATH)

random.seed(42)
exemplar_candidates = random.sample(tweets, 10)
exemplar_tweets = [x['tweet'] for x in exemplar_candidates]

In [3]:
# See the sampled tweets

for index, tweet in enumerate(exemplar_tweets): 
    text = show_document(index, tweet) 
    display(text) 

HTML(value="<h3 style='font-family: sans-serif; color:blue;'>Document 1:</h3><p style='font-family: Verdana'>G…

HTML(value="<h3 style='font-family: sans-serif; color:blue;'>Document 2:</h3><p style='font-family: Verdana'>E…

HTML(value="<h3 style='font-family: sans-serif; color:blue;'>Document 3:</h3><p style='font-family: Verdana'>O…

HTML(value="<h3 style='font-family: sans-serif; color:blue;'>Document 4:</h3><p style='font-family: Verdana'>F…

HTML(value="<h3 style='font-family: sans-serif; color:blue;'>Document 5:</h3><p style='font-family: Verdana'>T…

HTML(value="<h3 style='font-family: sans-serif; color:blue;'>Document 6:</h3><p style='font-family: Verdana'>🚨…

HTML(value="<h3 style='font-family: sans-serif; color:blue;'>Document 7:</h3><p style='font-family: Verdana'>@…

HTML(value="<h3 style='font-family: sans-serif; color:blue;'>Document 8:</h3><p style='font-family: Verdana'>I…

HTML(value="<h3 style='font-family: sans-serif; color:blue;'>Document 9:</h3><p style='font-family: Verdana'>S…

HTML(value="<h3 style='font-family: sans-serif; color:blue;'>Document 10:</h3><p style='font-family: Verdana'>…

### Step 2: Craft Implicit and Explicit Propositions
In this step, we craft both explicit and implicit exemplars for the sampled tweets. Fill up the textboxes next to the tweets with propositons and press the "Submit" button when you're done. 

##### TIP: 
If you want to start with the exemplars used in the paper, turn the ```start_with_existing_exemplars``` flag to ```True``` in the next cell.

Our sampled tweets and exemplars can also be found in  `exemplars/leg_tweets_exemplars.jsonl`

In [4]:
start_with_existing_exemplars = True

In [5]:
tweet_decomp_exemplars = []

if not start_with_existing_exemplars : 
    random.seed(42)
    exemplar_candidates = random.sample(tweets, 10)
    exemplar_tweets = [x['tweet'] for x in exemplar_candidates]

else: 
    # load exemplars used in the paper 
    paper_exemplars = read_jsonl("exemplars/leg_tweets_exemplars.jsonl")
    exemplar_tweets = [x[0] for x in paper_exemplars] 
    exemplar_decomps = [x[1] for x in paper_exemplars] 

for index, tweet in enumerate(exemplar_tweets[:5]): # remove slicing to include all tweets
    fancy_text = show_document(index, tweet)
    
    # display the document 
    display(fancy_text)
    
    if start_with_existing_exemplars: 
        decomps = create_textboxes("\n".join(exemplar_decomps[index]))
    else: 
        decomps = create_textboxes()
    
    tweet_decomp_exemplars.append([tweet, decomps])

HTML(value="<h3 style='font-family: sans-serif; color:blue;'>Document 1:</h3><p style='font-family: Verdana'>T…

Textarea(value='The Honest Ads Act will strengthen protections against foreign election interference\nRussia w…

Button(description='Submit', style=ButtonStyle())

HTML(value="<h3 style='font-family: sans-serif; color:blue;'>Document 2:</h3><p style='font-family: Verdana'>O…

Textarea(value="A police officer killed George Floyd\nGeorge Floyd's death was unjust\nBlack Americans deserve…

...

Textarea(value="Mark Zuckerberg makes insincere apologies\nFacebook is trying to avoid accountability\nFaceboo…

Button(description='Submit', style=ButtonStyle())

HTML(value="<h3 style='font-family: sans-serif; color:blue;'>Document 5:</h3><p style='font-family: Verdana'>F…

Textarea(value='There is an urgent need to help DACA recipients\nSupport for Dreamers is bipartisan\nDACA rece…

Button(description='Submit', style=ButtonStyle())

#### Step 3: Save them in the right format 

In [6]:
with open("exemplars/user_collected_exemplars.jsonl", "w") as f: 
    for elem in tweet_decomp_exemplars: 
        s= json.dumps(elem)
        f.write(f"{s}\n")

### Step 3: Prompting a LLM with the crafted exemplars

We use the `GenerationEmbedder` class from `eval_mteb.py` along with the hyperparameters specified in `configs/leg-tweet-gen-gpt3.5-propositions-all.yaml` to prompt GPT3.5 Turbo with the exemplars. The generated decompositions can be found in `data/gpt3.5_tweets_to_gen_all.jsonl`. 

Similar to **Step 2**, if you want to use the exemplars described in our paper, set `use_existing_exemplars = True` in the next cell. If you altered and submitted your own decompositionsin the previous cell, set `use_existing_exemplars=False`. 

**NOTE**: If you didn't submit anything in the last interactive shell, set `use_existing_exemplars` to `False`


In [19]:
use_existing_exemplars = True

In [20]:
from eval_mteb import  GenerationEmbedder, load_config

TWEETS_FILEPATH = Path('data/sampled_tweets_senate_115-117.jsonl')
tweets = read_jsonl(TWEETS_FILEPATH)

# load the config file and the exemplars 
config = load_config('configs/leg-tweet-gen-gpt3.5-propositions-all.yaml')

# use existing exemplars or 
if use_existing_exemplars is True: 
    exemplars = read_jsonl(config["data"]['exemplars_path'])
else: 
    exemplars = read_jsonl("exemplars/user_collected_exemplars.jsonl") 

# initialize the generation object with hyperparameters loaded from the config file
model = GenerationEmbedder(
    instructions=config["data"]["instructions"],
    openai_api_key=config["llm"]["openai_api_key"],
    exemplar_pool=exemplars,
    exemplar_format=config["exemplars"]["format"],
    exemplar_sep=config["exemplars"]["separator"],
    multi_output_sep=config["exemplars"]["multi_output_separator"],
    exemplars_per_prompt=config["exemplars"]["exemplars_per_prompt"],
    draws_per_pool=config["exemplars"]["draws_per_pool"],
    repeat_draws=config["exemplars"]["repeat_draws"],
    shuffles_per_draw=config["exemplars"]["shuffles_per_draw"],
    output_combination_strategy=config["embeddings"]["output_combination_strategy"],
    include_original_doc=config["embeddings"]["include_original_doc"],
    embedding_model_name=config["embeddings"]["embedding_model_name"],
    gen_model_name=config["llm"]["gen_model_name"],
    generations_per_prompt=config["llm"]["generations_per_prompt"],
    temperature=config["llm"]["temperature"],
    top_p=config["llm"]["top_p"],
    generation_kwargs=config["llm"]["generation_kwargs"],
    max_tokens=config["llm"]["max_tokens"],
    cache_db_path=config["main"]["cache_db_path"],
    dry_run=config["main"]["dry_run"],
    device=config["embeddings"]["device"],
    seed=config["main"]["seed"],
)

/fs/clip-political/rupak/miniconda3/envs/decompositions/lib/python3.10/site-packages/langchain/llms/openai.py:716: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


For the purpose of the tutorial, we are generating decompositions for the first 20 tweets which is stored in `outputs/test_outputs.jsonl`. Decompositions for the entire dataset can be found in `data/gpt3.5_tweets_to_gen_all.jsonl`.

In [23]:
# generate propositions from tweets 
# simple batching code that deals with breaks in connections

# use a small sample of tweets to test the generations

OUTPUT_PATH = Path("outputs/test_outputs.jsonl") 

# create a fresh file each time for testing outputs 
with OUTPUT_PATH.open(mode='w'): 
    pass 

In [24]:
if not OUTPUT_PATH.is_file():
    # If it doesn't exist, create the file
    OUTPUT_PATH.touch()

tweet_texts = [tweet['tweet'] for tweet in tweets][:20] # remove [:20] to run on all tweets
propositions = read_jsonl(OUTPUT_PATH)

batch_size = 10
for index in tqdm(range(len(propositions), len(tweet_texts), batch_size)):
    batch = tweet_texts[index:index+batch_size]
    propositions.extend(model.generate_from_inputs(batch))
    write_jsonl(propositions, OUTPUT_PATH)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:24<00:00, 12.38s/it]


#### Step 4: Validate

We sample some of the generated decompositions and confirm that they are _plausible_. In our paper, this was done using a human study. Please refer to Section 3 of our paper for more details. 


In [25]:
# before sampling, make sure to keep the tweet with the generations: 

for tweet_text, props in zip(tweet_texts, propositions):
    props.append(tweet_text)

# sample from the propositions
random.seed(42)
sample = random.sample(propositions, 5)

for elem in sample: 
    print(f"TWEET: \n{blue(elem[-1])}\n")
    print("PROPOSITONS:")
    for prop in elem[:-1]:
        print(green(prop))
    print("\n---------------------------\n")

TWEET: 
Ignoring #climatechange won't prevent devastating storms. People's lives depend on all of us acknowledging that #ClimateChangeIsReal  

PROPOSITONS:
Ignoring climate change won't prevent devastating storms
Climate change is a real and urgent issue
People's lives are at risk due to climate change
We need to acknowledge and address climate change

---------------------------

TWEET: 
We must do more to address mental health issues our veterans face and ensure all have access to treatment. @WSAZnews #suicidepreventionmonth  

PROPOSITONS:
Veterans face mental health issues
Access to mental health treatment for veterans is insufficient
More needs to be done to address mental health issues among veterans
Suicide prevention is important for veterans
Mental health support for veterans is lacking

---------------------------

TWEET: 
Elizabeth Cady Stanton. Lucretia Mott. Susan B. Anthony.
 
These strong women were true pioneers in the fight for women’s rights, &amp; they continue to i

#### Step 5: Use the propositions for your own downstream task!

## Finding tweets with implicit similarity

When we embed a document through the surface form of its content, documents with a similar communicative intent but are expressed differently in their lexical forms are placed further in the embedding space. 

To find such document pairs, we can make use of the inferential decompositions we obtained above. These decompositions helps us get over the lexical choices of a communicator, and lets us focus instead on their communicative intent. Hence, documents that seem far in the embedding space are brought closer through their similar decompositions. 

Here, we show some samples of such document pairs. 

In [26]:
from misc_utils import distance_func
# NOTE: distance_func is a simple distance function that returns the minimum pairwise distance among all possible inferences of a pair of tweets. 

# tweets
tweet_texts = [x['tweet'] for x in tweets] 

# load all the decompositions 
decompositions = read_jsonl("data/gpt3.5_tweets_to_gen_all.jsonl") 

assert len(tweet_texts) == len(decompositions), "Length of documents don't match length of decompositions" 

In [27]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
import torch
import numpy as np


st_model = SentenceTransformer("all-MiniLM-L6-v2") 
device = "cuda" if torch.cuda.is_available() else "cpu"

# compute embeddings and embedding similarities 
doc_embeddings = st_model.encode(tweet_texts, device=device, show_progress_bar=True) 
doc_distances = 1 - cos_sim(doc_embeddings, doc_embeddings) 

# compute embeddings of decompositions
# depending on your dataset, this might take a little bit of time
decomp_embeddings = np.array([st_model.encode(x, device=device) for x in tqdm(decompositions)], dtype="object")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1228 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39284/39284 [05:01<00:00, 130.25it/s]


In [28]:
# create a matrix of decomposition distances 
# try with only a subset of comments for speed 
doc_sample_size = 1000

from itertools import combinations 
pairs = list(combinations(range(doc_sample_size), 2)) 

decomp_dists = np.zeros((doc_sample_size, doc_sample_size)) 

for pair in tqdm(pairs): 
    index1, index2 = pair
    decomp_dists[index1, index2] = distance_func(decomp_embeddings[index1], decomp_embeddings[index2])[2]
    decomp_dists[index2, index1] = decomp_dists[index1, index2]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 499500/499500 [01:52<00:00, 4426.43it/s]


#### Finding tweet pairs that move closer in decomposition embedding space

In the next cell, we try to find pairs of documents (here, tweets) that move closer in embedding space once we look at their decompositions. The shift in distance is a hyperparameter, and best results can be obtained by setting a value that is meaningful for your dataset. 

Since we are using a distance function that looks at the minimum distance between all possible decompositions, the decomposition that brings the tweets closer is highlighted in <font color='red'>red</font>. The <font color='blue'>tweets</font> are colored in <font color='blue'>blue</font>, and <font color='green'>decompositions</font> in <font color='green'>green</font>.


In [29]:
# find pairs where distance is low(er) in decomposition space but high in comment space 
from nltk.tokenize import TweetTokenizer 
tt = TweetTokenizer() 

# change this value to control the number of examples displayed 
num_display_examples = 25
counter = 0 

for pair in tqdm(pairs): 
    if counter > num_display_examples: 
        break 
    
    index1, index2 = pair
    doc_distance = float(doc_distances[index1, index2])
    decomp_distance = decomp_dists[index1, index2] 
    #print(doc_distance, decomp_distance, doc_distance - decomp_distance) 
    
    # find docs of similar length
    if not abs(len(tt.tokenize(tweet_texts[index1])) - len(tt.tokenize(tweet_texts[index2]))) < 10: 
        continue 

    # find docs where the shift is more than 0.5 and distance over decomposition is low 
    # change these values accoring 
    if doc_distance - decomp_distance > 0.4 and decomp_distance < 0.3: 
        print(index1, index2)
        print(f"Distance moved from {doc_distance} -> {decomp_distance}")
        a = distance_func(decomp_embeddings[index1], decomp_embeddings[index2])
        
        print(blue(tweet_texts[index1]))
        print(green(decompositions[index1]))
        print(red(decompositions[index1][a[0]]))
    
        print(blue(tweet_texts[index2]))
        print(green(decompositions[index2]))
        print(red(decompositions[index2][a[1]]))
        print("**************")
        print("\n\n")
        counter += 1
        


  1%|██▊                                                                                                                                                                                              | 7437/499500 [00:02<03:20, 2455.33it/s]

7 20
Distance moved from 0.680652379989624 -> 0.1971825361251831
The cultural &amp; natural history of the @NewRiverNPS expands over 70,000 acres in WV. But we don’t currently have the funds necessary to maintain this beautiful WV landmark. I introduced a bill this wk to address the maintenance backlog at our national parks    
['New River National Park is an important part of cultural and natural history', 'The park requires funds to properly maintain it', "The park is an important part of West Virginia's heritage", 'Legislation has been introduced to address park maintenance', 'National Parks are an important part of American heritage']
National Parks are an important part of American heritage
Our #MonumentsForAll belong to all of us. They should not be sold off to the highest bidder. We will keep fighting to protect these special places which are a part of our history, our heritage, our local economies and our way of life.
['National monuments are important for American heritage', '

  2%|████▏                                                                                                                                                                                           | 11019/499500 [00:03<03:11, 2550.36it/s]

10 819
Distance moved from 0.5271098613739014 -> 0.11885702610015869
Thousands in DC will #MarchforLife today, the 45th anniversary of Roe v. Wade. As they march, they speak with one voice: "Life is sacred. Life is precious. Life is worth protecting." I stand with them &amp; all #ProLife Americans in defense of the unborn. #WhyWeMarch @March_for_Life
['Marchers believe that human life is sacred and valuable', 'Abortion should be illegal', 'The pro-life position is important', 'The March for Life is significant', 'Americans have a right to protest peacefully']
The March for Life is significant
I stand with students across #NM and America who are marching against gun violence, and look forward to joining the #MarchForOurLives in Santa Fe today. I pledge to do everything I can to enact common sense gun safety measures and end the gun lobby’s stranglehold over Washington.  
['Students are protesting gun violence across America', 'The March For Our Lives is an important event', 'Gun safety 

  3%|█████▍                                                                                                                                                                                          | 14242/499500 [00:04<02:54, 2778.30it/s]

...

48 378
Distance moved from 0.6953812837600708 -> 0.16763520240783691
Introduced the Keep Families Together &amp; Enforce the Law Act. This bill would #KeepFamiliesTogether during legal proceedings, protect children, authorize 225 more immigration judges &amp; ensure the integrity of our immigration laws.
['The Keep Families Together & Enforce the Law Act will protect immigrant families', 'Children deserve protection', 'More immigration judges are needed', 'Immigration laws must be upheld', 'The immigration system is in need of reform']
The immigration system is in need of reform
.@RepJayapal visited a federal prison and met with asylum seekers who had been transferred from the border. What they told her is horrifying. We need to fight back against Trump's cruel immigration policy.  
['Asylum seekers are being mistreated', 'A congresswoman visited a federal prison', "The Trump administration's immigration policies are cruel", 'Immigration policies need reform', 'Prison conditions in fed

 10%|██████████████████▊                                                                                                                                                                             | 48968/499500 [00:16<02:38, 2837.62it/s]

49 658
Distance moved from 0.9463708996772766 -> 0.28416907787323
More than 20 different government entities administer more than 160 different federal housing programs. I sent a letter asking for officials to identify duplication and overlap in federal housing assistance programs. 
['The federal government provides housing assistance through many different agencies', 'Housing programs may be unnecessarily duplicated', 'The government should reduce inefficiencies in federal housing programs', "Taxpayers' money should be spent wisely", 'A letter was sent to officials asking for a review of housing programs']
Taxpayers' money should be spent wisely
We spent over $170k to build trails in national parks. Seems like not too bad until you read the next line that the parks were in Russia
['Taxpayer money was used to build trails in Russian national parks', 'US funds should be used within the US', 'US money going to foreign countries can be problematic', 'The use of taxpayer funds should be cl

 10%|███████████████████▍                                                                                                                                                                            | 50663/499500 [00:17<02:34, 2897.65it/s]

52 83
Distance moved from 0.6601841449737549 -> 0.1399441957473755
New Hampshire, there are four more days left to #GetCovered. The Affordable Care Act open enrollment period ends December 15. Go to  today to get started. 
['The open enrollment period for the Affordable Care Act is closing soon', 'New Hampshire residents have four days left to sign up', 'Healthcare is important for all Americans', 'Healthcare costs should be affordable']
Healthcare is important for all Americans
🚨 Our fight to protect health care for millions of families is not over.  

Spread far and wide if useful. 
['A fight to protect healthcare has been undertaken', 'Millions of American families depend on the ACA for healthcare', 'Healthcare should be available for all Americans', 'We must keep fighting for healthcare']
Healthcare should be available for all Americans
**************



52 92
Distance moved from 0.5998044013977051 -> 0.1719595193862915
New Hampshire, there are four more days left to #GetCovered. T